In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,10854
2,Huelva,Confirmados PDIA 14 días,928
3,Huelva,Tasa PDIA 14 días,"181,69607806665974"
4,Huelva,Confirmados PDIA 7 días,508
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,2
629,Municipio de Huelva sin especificar,Total Confirmados,124
630,Municipio de Huelva sin especificar,Curados,101


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  10854.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3075.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 927 personas en los últimos 7 días 

Un positivo PCR cada 492 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,10854.0,508.0,928.0,510743.0,99.462939,181.696078,85.0
Huelva-Costa,6115.0,339.0,633.0,288115.0,117.661351,219.703938,66.0
Huelva (capital),3075.0,155.0,292.0,143663.0,107.891385,203.253447,30.0
Condado-Campiña,3333.0,150.0,262.0,155057.0,96.738619,168.970121,18.0
Cartaya,394.0,43.0,73.0,19974.0,215.279864,365.475118,9.0
Lepe,951.0,41.0,82.0,27431.0,149.465933,298.931865,8.0
Isla Cristina,418.0,36.0,89.0,21264.0,169.300226,418.547780,8.0
Aljaraque,429.0,36.0,57.0,21260.0,169.332079,268.109125,7.0
Palos de la Frontera,238.0,17.0,33.0,11289.0,150.589069,292.319957,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Palma del Condado (La),523.0,38.0,69.0,10761.0,353.127033,641.204349,4.0
Jabugo,46.0,10.0,12.0,2250.0,444.444444,533.333333,0.0
Lucena del Puerto,98.0,15.0,16.0,3371.0,444.971818,474.636606,3.0
Isla Cristina,418.0,36.0,89.0,21264.0,169.300226,418.547780,8.0
Cartaya,394.0,43.0,73.0,19974.0,215.279864,365.475118,9.0
Paterna del Campo,68.0,8.0,12.0,3509.0,227.985181,341.977771,0.0
Lepe,951.0,41.0,82.0,27431.0,149.465933,298.931865,8.0
Palos de la Frontera,238.0,17.0,33.0,11289.0,150.589069,292.319957,5.0
Aljaraque,429.0,36.0,57.0,21260.0,169.332079,268.109125,7.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villanueva de los Castillejos,61.0,1.0,5.0,2820.0,35.460993,177.304965,-1.0,0.200000
Villalba del Alcor,103.0,1.0,5.0,3338.0,29.958059,149.790294,0.0,0.200000
San Bartolomé de la Torre,45.0,1.0,5.0,3736.0,26.766595,133.832976,0.0,0.200000
Minas de Riotinto,67.0,1.0,3.0,3848.0,25.987526,77.962578,0.0,0.333333
Aroche,55.0,1.0,3.0,3073.0,32.541490,97.624471,0.0,0.333333
Isla Cristina,418.0,36.0,89.0,21264.0,169.300226,418.547780,8.0,0.404494
San Juan del Puerto,153.0,8.0,18.0,9300.0,86.021505,193.548387,0.0,0.444444
Valverde del Camino,273.0,2.0,4.0,12820.0,15.600624,31.201248,0.0,0.500000
Lepe,951.0,41.0,82.0,27431.0,149.465933,298.931865,8.0,0.500000
